# Powershell AMSI Bypass via .NET Reflection
Detects Request to amsiInitFailed that can be used to disable AMSI Scanning

## Rule Content
```
- title: Powershell AMSI Bypass via .NET Reflection
  id: 30edb182-aa75-42c0-b0a9-e998bb29067c
  status: experimental
  description: Detects Request to amsiInitFailed that can be used to disable AMSI
    Scanning
  references:
  - https://twitter.com/mattifestation/status/735261176745988096
  - https://www.hybrid-analysis.com/sample/0ced17419e01663a0cd836c9c2eb925e3031ffb5b18ccf35f4dea5d586d0203e?environmentId=120
  tags:
  - attack.execution
  - attack.defense_evasion
  - attack.t1086
  author: Markus Neis
  date: 2018/08/17
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      CommandLine:
      - '*System.Management.Automation.AmsiUtils*'
    selection2:
      CommandLine:
      - '*amsiInitFailed*'
    condition: selection1 and selection2
    falsepositives:
    - Potential Admin Activity
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:(*System.Management.Automation.AmsiUtils*) AND process_command_line.keyword:(*amsiInitFailed*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()